In [6]:
from itertools import repeat

import numpy as np

from helper_loader import *
from histalign.backend.ccf.paths import get_atlas_path
from histalign.backend.models import Orientation
from histalign.backend.registration.alignment import interpolate_sparse_3d_array

set_log_level("DEBUG")

In [2]:
def corrupt_random_noise(array: np.ndarray, corruption_rate: float = 0.9) -> np.ndarray:
    array = array.copy()

    array[
        np.where(
            np.random.choice(
                [0, 1],
                size=np.prod(array.shape),
                p=[1 - corruption_rate, corruption_rate],
            ).reshape(array.shape)
            > 0
        )
    ] = 0

    return array


def corrupt_slice(
    array: np.ndarray,
    corruption_rate: float = 0.9,
    regular: bool = False,
    orientation: Orientation = Orientation.SAGITTAL,
) -> np.ndarray:
    array = array.copy()

    match orientation:
        case Orientation.CORONAL:
            axis = 0
        case Orientation.HORIZONTAL:
            axis = 1
        case Orientation.SAGITTAL:
            axis = 2

    if regular:
        for i in range(array.shape[axis]):
            corrupt = not (i % (10 * corruption_rate) == 0)
            if corrupt:
                np.put_along_axis(array, np.array([[[i]]]), 0, axis=axis)
    else:
        for index, corrupt in enumerate(
            np.random.choice(
                [False, True],
                size=array.shape[axis],
                p=[1 - corruption_rate, corruption_rate],
            )
        ):
            if corrupt:
                np.put_along_axis(array, np.array([[[index]]]), 0, axis=axis)

    return array


def replace_workers_default(function: Callable, value: int) -> None:
    defaults = list(function.__defaults__)
    defaults[-1] = value
    function.__defaults__ = tuple(defaults)

In [4]:
array_path = get_atlas_path(Resolution.MICRONS_25)
array = load_volume(array_path, return_raw_array=True)

mask = load_structure_mask("Isocortex", Resolution.MICRONS_25)

masked_array = np.where(mask, array, 0)

corrupted_array = corrupt_slice(
    array,
    regular=True,
    corruption_rate=0.5,
    orientation=Orientation.SAGITTAL,
)
# imshow(corrupted_array[100], title="Corrupted array")

In [12]:
maes = {}
interpolated_arrays = []

for neighbours_count in [2**3, 3**3, 4**3]:
    interpolated_array = interpolate_sparse_3d_array(
        corrupted_array,
        reference_mask=mask,
        pre_masked=True,
        neighbours=neighbours_count,
    )

    maes[neighbours_count] = np.mean(
        np.abs(
            masked_array[masked_array.nonzero()]
            - interpolated_array[masked_array.nonzero()]
        )
    )
    interpolated_arrays.append(interpolated_array)

[2024-11-13 16:28:05] - [    INFO ] - Starting interpolation with parameters {kernel: multiquadric, neighbours: 8, epsilon: 1, degree: None, chunk size: 1,000,000, recursive: False}. (histalign.backend.registration.alignment:204)
[2024-11-13 16:28:10] - [    INFO ] - Interpolating chunk 1/8 (12%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:15] - [    INFO ] - Interpolating chunk 2/8 (25%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:19] - [    INFO ] - Interpolating chunk 3/8 (38%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:24] - [    INFO ] - Interpolating chunk 4/8 (50%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:28] - [    INFO ] - Interpolating chunk 5/8 (62%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:32] - [    INFO ] - Interpolating chunk 6/8 (75%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:36] - [    INFO ] - Interpolating chunk 7/8 (88%). (hist

Output:
```
[2024-11-13 16:28:05] - [    INFO ] - Starting interpolation with parameters {kernel: multiquadric, neighbours: 8, epsilon: 1, degree: None, chunk size: 1,000,000, recursive: False}. (histalign.backend.registration.alignment:204)
[2024-11-13 16:28:10] - [    INFO ] - Interpolating chunk 1/8 (12%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:15] - [    INFO ] - Interpolating chunk 2/8 (25%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:19] - [    INFO ] - Interpolating chunk 3/8 (38%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:24] - [    INFO ] - Interpolating chunk 4/8 (50%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:28] - [    INFO ] - Interpolating chunk 5/8 (62%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:32] - [    INFO ] - Interpolating chunk 6/8 (75%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:36] - [    INFO ] - Interpolating chunk 7/8 (88%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:40] - [    INFO ] - Interpolating chunk 8/8 (100%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:28:44] - [    INFO ] - Finished interpolation in 40s. (histalign.backend.registration.alignment:294)
[2024-11-13 16:28:47] - [    INFO ] - Starting interpolation with parameters {kernel: multiquadric, neighbours: 27, epsilon: 1, degree: None, chunk size: 1,000,000, recursive: False}. (histalign.backend.registration.alignment:204)
[2024-11-13 16:28:53] - [    INFO ] - Interpolating chunk 1/8 (12%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:29:22] - [    INFO ] - Interpolating chunk 2/8 (25%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:29:49] - [    INFO ] - Interpolating chunk 3/8 (38%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:30:17] - [    INFO ] - Interpolating chunk 4/8 (50%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:30:47] - [    INFO ] - Interpolating chunk 5/8 (62%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:31:16] - [    INFO ] - Interpolating chunk 6/8 (75%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:31:47] - [    INFO ] - Interpolating chunk 7/8 (88%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:32:17] - [    INFO ] - Interpolating chunk 8/8 (100%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:32:44] - [    INFO ] - Finished interpolation in  3m58s. (histalign.backend.registration.alignment:294)
[2024-11-13 16:32:46] - [    INFO ] - Starting interpolation with parameters {kernel: multiquadric, neighbours: 64, epsilon: 1, degree: None, chunk size: 1,000,000, recursive: False}. (histalign.backend.registration.alignment:204)
[2024-11-13 16:32:52] - [    INFO ] - Interpolating chunk 1/8 (12%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:36:02] - [    INFO ] - Interpolating chunk 2/8 (25%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:39:55] - [    INFO ] - Interpolating chunk 3/8 (38%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:43:42] - [    INFO ] - Interpolating chunk 4/8 (50%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:47:09] - [    INFO ] - Interpolating chunk 5/8 (62%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:51:04] - [    INFO ] - Interpolating chunk 6/8 (75%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:54:08] - [    INFO ] - Interpolating chunk 7/8 (88%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:56:59] - [    INFO ] - Interpolating chunk 8/8 (100%). (histalign.backend.registration.alignment:238)
[2024-11-13 16:59:46] - [    INFO ] - Finished interpolation in 27m 1s. (histalign.backend.registration.alignment:294)
```

In [13]:
maes

{8: 5.800753119125259, 27: 4.2247147727142185, 64: 3.6460130128908856}

Output:
```
{8: 5.800753119125259, 27: 4.2247147727142185, 64: 3.6460130128908856}
```

In [14]:
masked_array.max()

516

In [21]:
print(f"{(1 - maes[27] / maes[8]):.0%} reduction in error using 27 neighbours vs 8")
print(
    f"{(3 * 60 + 58) / 40:.0f} times increase in interpolation time using 27 neighbours vs 8"
)

27% reduction in error using 27 neighbours vs 8
6 times increase in interpolation time using 27 neighbours vs 8


```
27% reduction in error using 27 neighbours vs 8
6 times increase in interpolation time using 27 neighbours vs 8
```

In [22]:
print(f"{(1 - maes[64] / maes[8]):.0%} reduction in error using 64 neighbours vs 8")
print(
    f"{(27 * 60 + 1) / 40:.0f} times increase in interpolation time using 64 neighbours vs 8"
)

37% reduction in error using 64 neighbours vs 8
41 times increase in interpolation time using 64 neighbours vs 8


```
37% reduction in error using 64 neighbours vs 8
41 times increase in interpolation time using 64 neighbours vs 8
```